# Colorado Elk Data Processing

# Packages

In [2]:
import pandas as pd
import os

# Consoliate All Applicant Data

In [15]:
directory = 'Input-Data\\Applicant-Data\\'

All_Applicants = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")):
        df_Applicants = pd.read_excel(directory+file)
        All_Applicants = pd.concat([All_Applicants,df_Applicants])
        continue
    else:
        continue

del All_Applicants[All_Applicants.columns[0]]

All_Applicants['Secondary Key'] = [str(x) + "-" + y for x,y in zip(All_Applicants['Year'],All_Applicants['Hunt Code'])]

All_Applicants.to_excel('Output-Data\\All-Applicant-Preference_Points.xlsx')

# Import Applicant Data
Applicant_Data = pd.read_excel('Output-Data//All-Applicant-Preference_Points.xlsx')

# Consolidate All Draw Data

In [14]:
directory = 'Input-Data\\Draw-Data\\'

All_Draw = pd.DataFrame()

for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if (filename.endswith(".xlsx")):
        df_Draw = pd.read_excel(directory+file)
        All_Draw = pd.concat([All_Draw,df_Draw])
        continue
    else:
        continue

del All_Draw[All_Draw.columns[0]]

All_Draw['Secondary Key'] = [str(x) + "-" + y for x,y in zip(All_Draw['Year'],All_Draw['Hunt Code'])]

All_Draw.to_excel('Output-Data\\All-Draw-Preference_Points.xlsx')

# Import Draw Data
Draw_Data = pd.read_excel('Output-Data//All-Draw-Preference_Points.xlsx')

# Total Tags Awarded

In [16]:
#Isolate output for final join
Awarded_Final_Join = Draw_Data[['Hunt Code','Year','Secondary Key']].drop_duplicates('Secondary Key')

def TotalAwarded(InputColumnName,OutputColumnName):
    #Isolate columns of interest
    df_1 = Draw_Data[[InputColumnName,'Secondary Key']].copy()

    #Obtain the weighted total and number of tags issued
    df_2 = df_1.groupby('Secondary Key')[InputColumnName].sum().reset_index().copy()

    #Rename to Output Column
    df_3 = df_2.rename(columns={InputColumnName:OutputColumnName})

    #Isolate columns of interest
    OutputDF = df_3[['Secondary Key',OutputColumnName]]

    # Return the output dataframe
    return OutputDF

# Call functions to find the total number of tags awarded of each classification
Award_Adult_Res = TotalAwarded('D-Adult-Res','Adult-Res Awarded')
Award_Adult_NonRes = TotalAwarded('D-Adult-NonRes','Adult-NonRes Awarded')
Award_Youth_Res = TotalAwarded('D-Youth-Res','Youth-Res Awarded')
Award_Youth_NonRes = TotalAwarded('D-Youth-NonRes','Youth-NonRes Awarded')
Award_LPP_U = TotalAwarded('D-Landowner(LPP)-Unrestricted','LPP_U Awarded')
Award_LPP_R = TotalAwarded('D-Landownder(LPP)-Restricted','LPP_R Awarded')


#Merge All together
Output_Awarded = Awarded_Final_Join.merge(Award_Adult_Res, on='Secondary Key',how='left'\
    ).merge(Award_Adult_NonRes, on='Secondary Key',how='left'
    ).merge(Award_Youth_Res, on='Secondary Key',how='left'
    ).merge(Award_Youth_NonRes, on='Secondary Key',how='left'
    ).merge(Award_LPP_U, on='Secondary Key',how='left'
    ).merge(Award_LPP_R, on='Secondary Key',how='left')

#Sort by Hunt Code and Year, then reset index
Output_Awarded.sort_values(by=['Hunt Code','Year'],ascending=True,inplace=True)
Output_Awarded.reset_index(inplace=True,drop=True)

#Total number of Applicants
Output_Awarded['Total Awarded'] = [\
    a+b+c+d+e+f for a,b,c,d,e,f in zip(Output_Awarded['Adult-Res Awarded']
    ,Output_Awarded['Adult-NonRes Awarded'],Output_Awarded['Youth-Res Awarded']
    ,Output_Awarded['Youth-NonRes Awarded'],Output_Awarded['LPP_U Awarded']
    ,Output_Awarded['LPP_R Awarded'])]

#Output to Excel
Output_Awarded.to_excel('Output-Data\\Number-Awarded.xlsx')

# Total Applicants

In [17]:
#Isolate output for final join
Applicants_Final_Join = Applicant_Data[['Hunt Code','Year','Secondary Key']].drop_duplicates('Secondary Key')

def TotalApplicants(InputColumnName,OutputColumnName):
    #Isolate columns of interest
    df_1 = Applicant_Data[[InputColumnName,'Secondary Key']].copy()

    #Obtain the weighted total and number of tags issued
    df_2 = df_1.groupby('Secondary Key')[InputColumnName].sum().reset_index().copy()

    #Rename to Output Column
    df_3 = df_2.rename(columns={InputColumnName:OutputColumnName})

    #Isolate columns of interest
    OutputDF = df_3[['Secondary Key',OutputColumnName]]

    # Return the output dataframe
    return OutputDF

# Call functions to find the weighted average of each classification
App_Adult_Res = TotalApplicants('A-Adult-Res','Adult-Res Applicants')
App_Adult_NonRes = TotalApplicants('A-Adult-NonRes','Adult-NonRes Applicants')
App_Youth_Res = TotalApplicants('A-Youth-Res','Youth-Res Applicants')
App_Youth_NonRes = TotalApplicants('A-Youth-NonRes','Youth-NonRes Applicants')
App_LPP_U = TotalApplicants('A-Landowner(LPP)-Unrestricted','LPP_U Applicants')
App_LPP_R = TotalApplicants('A-Landownder(LPP)-Restricted','LPP_R Applicants')


#Merge All together
Output_Applicants = Applicants_Final_Join.merge(App_Adult_Res, on='Secondary Key',how='left'\
    ).merge(App_Adult_NonRes, on='Secondary Key',how='left'
    ).merge(App_Youth_Res, on='Secondary Key',how='left'
    ).merge(App_Youth_NonRes, on='Secondary Key',how='left'
    ).merge(App_LPP_U, on='Secondary Key',how='left'
    ).merge(App_LPP_R, on='Secondary Key',how='left')

#Sort by Hunt Code and Year, then reset index
Output_Applicants.sort_values(by=['Hunt Code','Year'],ascending=True,inplace=True)
Output_Applicants.reset_index(inplace=True,drop=True)

#Total number of Applicants
Output_Applicants['Total Applicants'] = [\
    a+b+c+d+e+f for a,b,c,d,e,f in zip(Output_Applicants['Adult-Res Applicants']
    ,Output_Applicants['Adult-NonRes Applicants'],Output_Applicants['Youth-Res Applicants']
    ,Output_Applicants['Youth-NonRes Applicants'],Output_Applicants['LPP_U Applicants']
    ,Output_Applicants['LPP_R Applicants'])]

#Output to Excel
Output_Applicants.to_excel('Output-Data\\Number-Of-Applicants.xlsx')

# Average Preference Points

In [18]:
#Isolate output for final join
Average_PP_1 = Draw_Data[['Hunt Code','Year','Secondary Key']].drop_duplicates('Secondary Key')


def WeightedAverage(InputColumnName,OutputColumnName):
    #Isolate columns of interest
    df_1 = Draw_Data[['Preference Points',InputColumnName,'Secondary Key']].copy()

    #Find the weighted average
    df_1['Weighted Total'] = df_1['Preference Points'] * df_1[InputColumnName]

    #Obtain the weighted total and number of tags issued
    df_2 = df_1.groupby('Secondary Key')['Weighted Total',InputColumnName].sum().reset_index().copy()

    #Determine the weighted average
    df_2[OutputColumnName] = df_2['Weighted Total'] / df_2[InputColumnName]

    #Isolate columns of interest
    OutputDF = df_2[['Secondary Key',OutputColumnName]]

    # Return the output dataframe
    return OutputDF

# Call functions to find the weighted average of each classification
AvgPP_Adult_Res = WeightedAverage('D-Adult-Res','Adult-Res Average')
AvgPP_Adult_NonRes = WeightedAverage('D-Adult-NonRes','Adult-NonRes Average')
AvgPP_Youth_Res = WeightedAverage('D-Youth-Res','Youth-Res Average')
AvgPP_Youth_NonRes = WeightedAverage('D-Youth-NonRes','Youth-NonRes Average')
AvgPP_LPP_U = WeightedAverage('D-Landowner(LPP)-Unrestricted','LPP_U Average')
AvgPP_LPP_R = WeightedAverage('D-Landownder(LPP)-Restricted','LPP_R Average')


#Merge the Adult Residents
Average_PP_2 = Average_PP_1.merge(AvgPP_Adult_Res, on='Secondary Key',how='left'\
    ).merge(AvgPP_Adult_NonRes, on='Secondary Key',how='left'
    ).merge(AvgPP_Youth_Res, on='Secondary Key',how='left'
    ).merge(AvgPP_Youth_NonRes, on='Secondary Key',how='left'
    ).merge(AvgPP_LPP_U, on='Secondary Key',how='left'
    ).merge(AvgPP_LPP_R, on='Secondary Key',how='left')

#Sort by Hunt Code and Year, then reset index
Average_PP_2.sort_values(by=['Hunt Code','Year'],ascending=True,inplace=True)
Average_PP_2.reset_index(inplace=True,drop=True)

#Output to Excel
Average_PP_2.to_excel('Output-Data\\Average-Preference_Points.xlsx')

C:\Users\clanc\AppData\Local\Temp\ipykernel_13988\2502123941.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_2 = df_1.groupby('Secondary Key')['Weighted Total',InputColumnName].sum().reset_index().copy()
C:\Users\clanc\AppData\Local\Temp\ipykernel_13988\2502123941.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_2 = df_1.groupby('Secondary Key')['Weighted Total',InputColumnName].sum().reset_index().copy()
C:\Users\clanc\AppData\Local\Temp\ipykernel_13988\2502123941.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_2 = df_1.groupby('Secondary Key')['Weighted Total',InputColumnName].sum().reset_index().copy()
C:\Users\clanc\AppData\Local\Temp\ipykernel_13988\2502123941.py:13: FutureWarning: Indexing with multiple keys (implicitl

# Minimum Preference Points

In [19]:
#Isolate output for final join
Min_PP_1 = Draw_Data[['Hunt Code','Year','Secondary Key']].drop_duplicates('Secondary Key')

def MinimumPoints(InputColumnName,OutputColumnName):
    #Isolate columns of interest
    df_1 = Draw_Data[['Preference Points',InputColumnName,'Secondary Key']].copy()

    #Isolate Preference points that had a tag associated with them
    df_2 = df_1.loc[df_1[InputColumnName] > 0]
 
    #Obtain the weighted total and number of tags issued
    df_3 = df_2.groupby('Secondary Key')['Preference Points'].min().reset_index().copy()

    #Determine the weighted average
    df_3[OutputColumnName] = df_3['Preference Points']

    #Isolate columns of interest
    OutputDF = df_3[['Secondary Key',OutputColumnName]]

    # Return the output dataframe
    return OutputDF

# Call functions to find the weighted average of each classification
MinPP_Adult_Res = MinimumPoints('D-Adult-Res','Adult-Res Min')
MinPP_Adult_NonRes = MinimumPoints('D-Adult-NonRes','Adult-NonRes Min')
MinPP_Youth_Res = MinimumPoints('D-Youth-Res','Youth-Res Min')
MinPP_Youth_NonRes = MinimumPoints('D-Youth-NonRes','Youth-NonRes Min')
MinPP_LPP_U = MinimumPoints('D-Landowner(LPP)-Unrestricted','LPP_U Min')
MinPP_LPP_R = MinimumPoints('D-Landownder(LPP)-Restricted','LPP_R Min')


#Merge the Adult Residents
Min_PP_2 = Min_PP_1.merge(MinPP_Adult_Res, on='Secondary Key',how='left'\
    ).merge(MinPP_Adult_NonRes, on='Secondary Key',how='left'
    ).merge(MinPP_Youth_Res, on='Secondary Key',how='left'
    ).merge(MinPP_Youth_NonRes, on='Secondary Key',how='left'
    ).merge(MinPP_LPP_U, on='Secondary Key',how='left'
    ).merge(MinPP_LPP_R, on='Secondary Key',how='left')

#Sort by Hunt Code and Year, then reset index
Min_PP_2.sort_values(by=['Hunt Code','Year'],ascending=True,inplace=True)
Min_PP_2.reset_index(inplace=True,drop=True)

#Output to Excel
Min_PP_2.to_excel('Output-Data\\Minimum-Preference_Points.xlsx')